In [1]:
!pip install pytorch2keras

In [2]:
!pip install onnx==1.8.1

In [3]:
from pytorch2keras import pytorch_to_keras

In [5]:
import torch
import torch.nn as nn

In [6]:
class TestConv2d(nn.Module):
    """
    Module for Conv2d testing
    """

    def __init__(self, inp=10, out=16, kernel_size=3):
        super(TestConv2d, self).__init__()
        self.conv2d = nn.Conv2d(inp, out, stride=1, kernel_size=kernel_size, bias=True)

    def forward(self, x):
        x = self.conv2d(x)
        return x

model = TestConv2d()

# load weights here
# model.load_state_dict(torch.load(path_to_weights.pth))

In [7]:
from torch.autograd import Variable
import numpy as np

input_np = np.random.uniform(0, 1, (1, 10, 32, 32))
input_var = Variable(torch.FloatTensor(input_np))

In [10]:
k_model = pytorch_to_keras(model, input_var, [(10, 32, 32,)], verbose=True, name_policy='short')  

INFO:pytorch2keras:Converter is called.
DEBUG:pytorch2keras:Input_names:
DEBUG:pytorch2keras:['input_0']
DEBUG:pytorch2keras:Output_names:
DEBUG:pytorch2keras:['output_0']
INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:[(10, 32, 32)]
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> input_0.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> output_0.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight conv2d.weight with shape (16, 10, 3, 3).
DEBUG:onnx2keras:Found weight conv2d.bias with shape (16,).
DEBUG:onnx2keras:Found input input_0 with shape (10, 32, 32)
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Conv
DEBUG:onnx2keras:node_name: output_0
DEBUG:onnx2keras:node_params: {'dilations': [1, 1], 'group': 1, 'kernel_shape': [3, 3], 'pads': [0, 0, 0, 0], 'strides': [1, 1], 'change_ordering': False, 'name_policy': 'short'}
DEBUG:onnx2k

graph(%input_0 : Float(1, 10, 32, 32, strides=[10240, 1024, 32, 1], requires_grad=0, device=cpu),
      %conv2d.weight : Float(16, 10, 3, 3, strides=[90, 9, 3, 1], requires_grad=1, device=cpu),
      %conv2d.bias : Float(16, strides=[1], requires_grad=1, device=cpu)):
  %output_0 : Float(1, 16, 30, 30, strides=[14400, 900, 30, 1], requires_grad=1, device=cpu) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[3, 3], pads=[0, 0, 0, 0], strides=[1, 1]](%input_0, %conv2d.weight, %conv2d.bias) # /usr/local/lib/python3.7/dist-packages/torch/nn/modules/conv.py:443:0
  return (%output_0)



In [12]:
%tensorflow_version 2.3.0

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.3.0`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [14]:
import tensorflow
print(tensorflow.__version__)

2.7.0


In [11]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = \
            list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph

from keras import backend as K
import tensorflow as tf
frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in k_model.outputs])

tf.train.write_graph(frozen_graph, ".", "my_model.pb", as_text=False)
print([i for i in k_model.outputs])

TypeError: ignored

In [ ]:
# You will see the output layer name, so, now it's time to convert my_model.pb to tfjs model:

# tensorflowjs_converter  \
#     --input_format=tf_frozen_model \
#     --output_node_names='TANHTObs/Tanh' \
#     my_model.pb \
#     model_tfjs

# Thats all!

# const MODEL_URL = `model_tfjs/tensorflowjs_model.pb`;
# const WEIGHTS_URL = `model_tfjs/weights_manifest.json`;
# cont model = await tf.loadFrozenModel(MODEL_URL, WEIGHTS_URL);